# Corretor Ortográfico

Sempre quando queremos aprender novos conhecimentos, buscamos mais informações sobre o assunto em livros, vídeos, cursos, palestras, aulas e etc. Para que nosso modelo corrija as palavras, precisaremos primeiro ensiná-lo a escrever através de um vocabulário que cresce conforme aprende.

Logo, iremos precisar de uma base de dados.

Porém, como estamos trabalhando com o buscador do site da Alura, seria interessante também que essa base de conhecimento tivesse termos mais técnicos da área.

A base de dados que utilizaremos neste curso será construída com os próprios artigos do Blog da Alura, pois ensinaremos o nosso corretor a realizar correções específicas para o mundo técnico de desenvolvimento, como Java, programação orientada a objeto, Data Science e etc.

Quando trabalhamos em NLP, a nossa base de dados é conhecida como corpus. Ou seja, é um corpo composto por com diversos textos, e cada um corresponde a um artigo do nosso blog, chamado de documento.

Logo, o corpus é um conjunto de documentos em Processamento Linguagem Natural, e no nosso caso, é composto pelos artigos do blog que formam a base de dados artigos.txt. Em seguida, a leremos no notebook.


In [1]:
with open("corpus/artigos.txt", "r", encoding = "utf-8") as f:
    artigos = f.read()

print(artigos[:500])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:

java 

Suponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou


Imaginemos que dois estudantes estão aprendendo um novo idioma, o russo.

Ao final de um ano de estudos, o primeiro leu apenas um livro de 100 páginas, enquanto o outro leu toda a coleção do Game of Thrones em russo, sendo que cada parte possui quase mil páginas. Neste caso, a segunda pessoa possui muito mais informações sobre a língua, pois teve acesso à um vocabulário muito maior, inclusive estando apta a fazer correções.

Portanto, a quantidade de vocábulos é interessante para este aprendizado, o que também vale para o nosso corretor.

Para sabermos se o nosso corpus é ideal para o trabalho, deveremos saber quantas palavras possui. Afinal, um texto com dez palavras faz no máximo dez correções por exemplo. Então precisaremos ter um corpus com um grande volume de termos.


In [2]:
len(artigos)

2605046

Neste caso, utilizar apenas a função len() recebendo artigos não nos ajudará, pois nos retornará apenas a quantidade de caracteres presentes no corpus, e não a de palavras que queremos.

In [3]:
texto_exemplo = "Olá, tudo bem?"
tokens = texto_exemplo.split()

In [4]:
print(len(tokens))

3


In [5]:
print(tokens)

['Olá,', 'tudo', 'bem?']


### Separando Palavras de Tokens

Porém, nem a vírgula da primeira parte em 'Olá,' e nem o ponto de interrogação na terceira em 'bem?' fazem parte dessas palavras em si.

De qualquer forma o método split() pode nos ajudar neste caso. Se imprimirmos o len() de tokens, teremos o tamanho do vetor com os três itens retornados no comando anterior.

Mas queremos os vocábulos separados, e não concatenados com pontuação.

Em Processamento de Linguagem Natural, cada um dos elementos separados desse vetor é conhecido como token. Ou seja, ao pegarmos as strings e as separarmos em pequenos pedaços, estamos tokenizando a frase.

Portanto, ainda temos apenas tokens separados, e não palavras separadas propriamente ditas.

Uma ferramenta bem conhecida na área de NLP é o nltk ou Natural Language Tool Kit, que é um conjunto de ferramentas que implementa diversos métodos e algoritmos para análise textual.



In [6]:
!pip install nltk

In [7]:
import nltk

In [9]:
nltk.download('punkt')
palavras_separadas = nltk.tokenize.word_tokenize(texto_exemplo)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaoe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [10]:
palavras_separadas

['Olá', ',', 'tudo', 'bem', '?']

In [11]:
len(palavras_separadas)

5

Ou seja, a frase "Olá, tudo bem?" possui cinco tokens, sendo três palavras e duas pontuações, mas precisamos descobrir apenas a quantidade de vocábulos no corpus.

Para fazermos a separação dos tokens, criaremos uma função 


In [14]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():             #isalpha() selecionará apenas as letras do alfabeto, retornando-as como verdadeiro
            lista_palavras.append(token)
    return lista_palavras

separa_palavras(palavras_separadas)

['Olá', 'tudo', 'bem']

Então, vamos aplicar este método em nosso corpus.

In [16]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
print(f"O número de palavras é {len(lista_palavras)}")

O número de palavras é 403106


O valor total de palavras retornado não representa a quantidade de palavras únicas que podem ser corrigidas, afinal muitas delas aparecerão *repetidas *nos textos.

Portanto, precisaremos calcular quantos vocábulos únicos existem sem repetição.

### Normalização

In [20]:
def normalizacao(lista_palavras):      # Esta função retorna uma lista com as letras todas em minuscúlas.
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
print(lista_normalizada[:10])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


In [21]:
len(set(lista_normalizada))        # set() remove as repetições de palavras

18465

Este será o número de palavras que nosso corretor ortográfico aprenderá a corrigir

## Desenvolvendo e testanto o corretor

### Inserindo uma letra

Usando como exemplo a palavra digitada errada " Lgica " onde o certo seria " Lógica ".

Nosso modelo de NLP precisará de um algoritmo capaz de inserir uma letra a mais neste termo digitado equivocadamente.



| ESQUERDO | DIREITO |     OPERAÇÃO     | RESULTADO |
|:--------:|:-------:|:----------------:|:---------:|
| null     | lgica   | null + ó + lgica | óligca    |
| l        | gica    | l + ó + gica     | lógica    |
| lg       | ica     | lg + ó + ica     | lgóica    |
| lgi      | ca      | lgi + ó + ca     | lgióca    |
| lgic     | a       | lgic + ó + a     | lgicóa    |
| lgica    | null    | lgica + ó + null | lgicaó    |

In [42]:
palavra_exemplo = "lgica"

def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas)

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'àlgica', 'álgica', 'âlgica', 'ãlgica', 'èlgica', 'élgica', 'êlgica', 'ìlgica', 'ílgica', 'îlgica', 'òlgica', 'ólgica', 'ôlgica', 'õlgica', 'ùlgica', 'úlgica', 'ûlgica', 'çlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'làgica', 'lágica', 'lâgica', 'lãgica', 'lègica', 'légica', 'lêgica', 'lìgica', 'lígica', 'lîgica', 'lògica', 'lógica', 'lôgica', 'lõgica', 'lùgica', 'lúgica', 'lûgica', 'lçgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica',

Como saída de palavras_geradas, teremos todas as palavras geradas por inserção.

Precisaremos de uma função que indicará 'lógica' como a palavra correta dentre as demais.

Temos um arquivo de texto, e queremos verificar se um termo está correto ou não de acordo com sua existência em nossa base de dados. Também deveremos calcular suas probabilidades dentro do conjunto de palavras geradas, pegando a correta como a de máxima probabilidade.

Pra construirmos a função probabilidade() em uma nova célula, primeiro precisaremos fazer este cálculo com uma palavra; a frequência do termo “lógica” será a quantidade de vezes que este parece no corpus em relação ao total de palavras. Para calcular, o nltk possui a função .FreqDist().

In [43]:
frequencia = nltk.FreqDist(lista_normalizada)
frequencia.most_common(10)

[('de', 15502),
 ('o', 14056),
 ('que', 12230),
 ('a', 11099),
 ('e', 10501),
 ('para', 7710),
 ('um', 6368),
 ('é', 5899),
 ('uma', 5220),
 ('do', 5124)]

In [44]:
frequencia["lógica"]

96

In [45]:
total_palavras = len(lista_normalizada)

In [46]:
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

probabilidade("lógica")

0.00023815075935361914

In [47]:
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [49]:
corretor(palavra_exemplo)

'lógica'

A saída do corretor será justamente a palavra 'lógica' mais frequente, a qual é a correção para quando digitamos por exemplo “lgica”. 

Porém, só estamos corrigindo o tipo de erro onde uma letra não foi digitada, e é interessante tratarmos outros casos.